In [40]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Training Deep Neural Networks

Training a deep DNN isn't a walk in the park. Here are some of the problems you could run into: <br>
1. You many be faced with the trickey *vanishing gradients* problem or the related *exploding gradients* problem. This is when the gradients grow smaller and smaller, or larger and larger, when flowing backward through the DNN during training. Both of these problems make lower layers very hard to train. <br><br>
2. You might not have enough training data for such a large network, or it might be too costly to label. <br><br>
3. Training may be extremely slow. <br><br>
4. A model with millions of parameters would severely risk overfitting the training set, especially if there are not enough training instances or if the yare too noisy.

## The Vanishing/Exploding Gradients Problems

Unfortunately, during backpropagation, gradients often get smaller and smaller as the algorithm progresses down to the lower layers. As a result, the Gradient Descent update leaves the lower layers' connection weights virtually unchanged, and the training never converges to a good solution. We call this the *vanishing gradients* problem. More generally, deep neural networks suffer from unstable gradients; different layers may learn at widely different speeds.

This unfortunate behavior was empirically observed long ago, and it was one of the reasons deep neural networks were mostly abandoned in the early 2000's, but some light was shed in a 2010 paper by Xavier Glorot and Yoshua Bengio. The authors found a few suspects, including the combination of the popular logistic sigmoid activation function and the weight initialization technique that was most popular at the time (ie.e, a normal distribution with a mean of 0 and a standard deviation of 1).

In short, they showed that with the activation function and this initialization scheme, the variance of the outputs of each layer is much greater than the variance of its inputs.

### Glorot and He Initialization

In their paper, Glorot and Bengio propose that we need the signal to flow properly in both directions: in the forward direction when making predictions, and in the reverse direction when backpropagating gradients. We don't want the signal to die out, nor do we want it to explode and saturate. For the signal to flow properly, the authors argue that we need the variance of the outputs of each layer to be equal to the variance of its inputs, and we need the gradients to have equal variance before and after flowing through a layer in the reverse direction. It is actually not possible to guarantee both, but Glorot and Bengio proposed *Xavier initialization or Glorot initialization*

Here's an analogy: if you set a microphone amplifier's knob too close to zero, people won't hear your voice, but if you set it too close to the max, your voice will be saturated and people won't understand what you're saying. Now imagine a chain of such amplifiers: they all need to be set properly in order for your voice to come out loud and clear at the end of the chain. Your voice has to come out of each amplifier at the same amplitude as it came in.

Using Glorot initialization can speed up training considerably, and it is one of the tricks that led to the success of Deep Learning. By default, Keras uses Glorot initialization with a uniform distribution. When creating a layer, you can change this to He initialization by setting **kernel_initialzier='he_uniform'** or **kernel_initializer='he_normal'** like below.

If you want He initialization with a uniform distribution but ased on $fan_{avg}$ rather than $fan_{in}$ you can use the VarianceScaling initializer as well.

In [35]:
init_helper = pd.DataFrame(
    columns=['Initialization', 'Activation Functions', '$\sigma^{2}$ (Normal)'],
    data=np.array([
        ['Glorot', 'None, tanh, logistic, softmax', r'$\frac{1}{fan_{avg}}$'],
        ['He', 'ReLU and variants', r'$\frac{2}{fan_{in}}$'],
        ['LeCun', 'SELU', r'$\frac{1}{fan_{in}}$']
    ])
)

init_helper

,Initialization,Activation Functions,$\sigma^{2}$ (Normal)
0,Glorot,"None, tanh, logistic, softmax",$\frac{1}{fan_{avg}}$
1,He,ReLU and variants,$\frac{2}{fan_{in}}$
2,LeCun,SELU,$\frac{1}{fan_{in}}$


In [44]:
# He Normal
he_norm = tf.keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

# He using fan avg
avg_init = tf.keras.initializers.VarianceScaling(
    scale=2,
    mode='fan_avg',
    distribution='uniform'
)
he_avg_init = tf.keras.layers.Dense(10, activation='relu', kernel_initializer=avg_init)

### Nonsaturating Activation Functions

One of the insights in the 2010 paper by Glorot and Bengio was that the problems with unstable gradients were in part due to a poor choice of activation functions. It turns out that other activations behave much better in deep neural networks-- in particular, the ReLU activation function. 

Unfortunately, the ReLU activiation function is not perfect. It suffers from a problem known as the *dying ReLUs*: during training, some neurons effectively "die", meaning they stop outputting anything other than 0. To solve this problem, you may want to use a variant of the ReLU function such as the *leaky ReLU*. The hyperparameter $\alpha$ defines how much the function "leaks": it is the slope of the function for z < 0 and is typically set to 0.01.

A 2015 paper compared several variants of the ReLU activiation function, and one of its conclusions was that the leaky variants always outperformed the strict ReLU activation function. In fact, setting $\alpha$ = 0.2 (a huge leak) seemed to result in better performance than $\alpha$ = 0.01 (a small leak).

*Randomized leaky ReLU (RReLU)*, where $\alpha$ is selected at random, performed fairly well and seemed to act as a regularizer. Finally, the paper evaluated the *parametric leaky ReLU (PReLU)*, where $\alpha$ is authorized to be learned during training (instead of being a hyperparameter, it becomes a parameter that can be modified by backpropagation like any other parameter). **PReLU was reported to strongly outperform ReLU on large image datasets, but on smaller datasets it runs the risk of overfitting the training set.**

Last but not least, **a 2015 paper proposed a new activation function called the *exponential linear unit (ELU)* that outperformed all the ReLU variants** in the authors' experiments: training time was reduced and the neural network performed better on the test set.

The ELU activation function looks a lot like the ReLU function, with a few major differences: <br>
1. It takes on negative values when z < 0, which allows the unit to have an average output closer to 0 and helps alleviate the vanishing gradients problem. The hyperparameter $\alpha$ defines the value that the ELU function approaches when z is a large negative number. It is usually set to 1, but you can tweak it like any other hyperparameter. <br><br>
2. It has a nonzero gradient for z < 0, which avoids the dead neurons problem. <br><br>
3. If $\alpha$ is equal to 1 then the function is smooth everywhere, including around z = 0, which helps speed up Gradient Descent since it does not bounce as much to the left and right of z = 0

**The main drawback of the ELU activiation function is that it is slower to compute than the ReLU function and its variants** (due to the use of the exponential function). Its faster convergence rate during training compensates for that slow computation, but still, at test time an ELU network will be slower than a ReLU network.

Then, a 2017 paper introduced the *Scaled ELU (SELU)* activation function: as its named suggests, it is a scaled variant of the ELU activation function. The authors showed that if you build a neural network composed exclusively of a stack of dense layers, and if all hidden layers use the SELU activation function, then the network will *self-normalize*: the output of each layer will tend to preserve a mean of 0 and a standard deviation of 1 during training, which solves the vanishing/exploding gradients problem. There are, however, a few conditions for self-normalization to happen: <br>
1. The input features must be standardized (mean 0 and standard deviations 1) <br><br>
2. Every hidden layer's weights must be initialized with LeCun normal initialization. In Keras, this means setting kernel_initializer='lecun_normal'<br><br>
3. The network's architecture must be sequential. **Unfortunately, if you try to use SELU in nonsequential architectures, such as recurrent networks or networks with skip connections (i.e. connections that skip layers, such as in Wide & Deep nets), self-normalization will not be guaranteed, so SELU will not necessarily outperform other activation functions.**<br><br>
4. The paper only guarantees self-normalization if all layers are dense, but some researchers have noted that the SELU activation function can improve performance in convolutional neural nets as well.

***So, which activation should you use for the hidden layers of your deep neural networks?*** Although your mileage will vary, in general **SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic**. 

If the network's architecture prevents its from self-normalizing, then ELU may perform better than SELU (since SELU is not smooth at z = 0). If you care a lot about runtime latency, then you may prefer leaky ReLU. If you don't want to tweak yet another hyperparameter, you may use the default $\alpha$ values used by Keras (e.g., 0.3 for leaky ReLU). If you ahve spare time and computing power, you can use cross-validation to evaluate other activation functions, such as RReLU if your network is overfitting or PReLU if you have a huge training set. **That said, because ReLU is the most used activation function (by far), may libraries and hardware accelerators provide ReLU-specific optimizations; therefore, if speed is your priority, ReLU might still be the best choice.**

In [55]:
# Example of a model using LeakyReLU
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, kernel_initializer='he_normal'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(1)
])

# Example of a model using PReLU
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, kernel_initializer='he_normal'),
    tf.keras.layers.PReLU(),
    tf.keras.layers.Dense(1)
])

# Examlpe of a model using SELU
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.Dense(1)
])

### Batch Normalization

Although using He initialization along with ELU (or any variant of ReLU) can significantly reduce the danger of the vanishing/exploding gradients problems at the beginning of training, it doesn't guarantee that they won't come bak during training. *Batch Normalization (BN)* addresses these problems. The technique consists of adding an operation in the model just before or after the activation function of each hidden layer. **This operation simply zero-centers and normalizes each input, then scales and shifts the result using two new parameter vectors per layer: one for scaling, the other for shifting.**

In other words, the operation lets the model learn the optimal scale and mean of each of the layer's inputs. In many cases, if you add a BN layer as the very first layer of your neural network, you do not need to standardize your training set (e.g. using StandardScaler); the BN layer will do it for you (well, approximately, since it only looks at one batch at a time, and it can also rescale and shift each input feature).

Unfortunatley, its not that simple. We may need to make predictions for individuals instances rather than for batches of instances. Moreover, even if we do have a batch of instances, it may be too small, or the instances may not be independent and identically distributed, so computing statistics over the batch instances would be unreliable. 

One solution could be to wait until the end of training, then run the whole training set through the neural network and compute the mean and standard deviation of each input of the BN layer. These "final" input means and standard deviations could then be used instead of the batch input means and standard deviations when making predictions. **However, most implementations of Batch Normalization estimate these final statistics during training by using a moving average of the layer's input means and standard deviations. This is what Keras does automatically when you use the BatchNormalization layer.**

Ioffe and Szegedy demonstrated that Batch Normalization considerably improved all the deep neural networks they experiments with. The vanishing gradients problem was strongly reduced to the point that they could use saturating activation functions such as the tanh and even logistic activation function. The networks were also much less sensitive to the weight initialization. The authors were able to use much larger learning rates as well.

Specifically they note that:
    * Applied to a state-of-the-art image classification model, Batch Normalization achieves the same accuracy with 14 times fewer training steps, and beats the original model by a significant margin. Using an ensemble of batch-normalized networks, we improve upon the best published result of ImageNet classification: reaching 4.9% top-5 validation error (4.8% test error), exceeding the accuracy of human raters.
    
Finally, like a gift that keeps on giving, Batch Normalization acts like a regularizer, reducing the need for other regularization techniques (such as dropout). 

Batch Normalization does, however, add some complexity to the model. Moreover, there is a runtime penalty: the neural network makes slower predictions due to the extra computations required at each layer. **Fortunatley, it's often possible to fuse the BN layer with the previous layer, after training, thereby avoiding the runtime penalty.**

You may find that training is rather slow, because each epoch takes much more time when you use Batch Normalization. This is usually counterbalanced by the fact that convergence is much faster with BN, so it will take fewer epochs to reach the same performance. All in all, *wall time* will usually be shorter.

#### Implementing Batch Normalization with Keras

As with most things with Keras, implementing Batch Normalization is simple and intuitive. Just add a BatchNormalization layer before or after each hidden layer's activation function, and optionally add a BN layer as well as the first layer in your model. The authors of the BN paper argued in favor of adding the BN layers before the activation functions, rather than after. You can experiment with this too to see which option works best on your dataset. To add the BN layers before the activation functions, you must remove the activation function from the hidden layers and add them as separate layers after the BN layers. Moreover, since a Batch Normalization layer includes one offset parameter per input, you can remove the bias term from the previous layer (just pass use_bias=False when creating it).

The BatchNormalization class has quite a few hyperparameters you can tweak. The defaults will usually be fine, but you may occasionally need to tweak the momentum. This hyperparameter is used by the BatchNormalization layer when it updates the exponential moving averages; given a new value **v**. A good momentum value is typically close to 1; for example, 0.9, 0.99, or 0.999 (you want more 9s for larger datsets and smaller mini-batches). Another important hyperparameter is axis. It defaults to -1, meaning that by default it will normalize the last axis. When the input batch is 2D (i.e., the batch shape is [batch size, features]), this means that each input feature will be normalized based on the mean and standard deviation computer across all the instances in the batch. If we move the first BN layer before the Flatten layer, then the input batches will be #D, with shape [batch size, height, width]; therefore, the BN layer will computer 28 means and 28 standard deviations (1 per column of pixels, computed across all instances in the batch and across all rows in the column), and it will normalize all pixels in a given column using the same mean and standard deviation. There will also be just 28 sclare parameters and 28 shift parameters. If instead you still want to treat each of the 784 pixels independently, then you should set axis=[1, 2].

BatchNormalization has become one of the most-used layers in deep neural networks, to the point that it is often omitted in the diagrams, as it is assumed that BN is added after every layer. But a recent paper may change this assumption: by using a novel *fixed-update* weight initialization technique. The authors managed to traing a very deep neural network (10,000 layers!) without BN, achieving state-of-the-art performance on complex image classification tasks. As this is bleeding-edge research, however, you may want to wait for additional research to confirm this finding before you drop Batch Normalization.

In [69]:
# Example applying BN after activation functions
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

print([(var.name, var.trainable) for var in model.layers[1].variables])
model.summary()

[('batch_normalization_7/gamma:0', True), ('batch_normalization_7/beta:0', True), ('batch_normalization_7/moving_mean:0', False), ('batch_normalization_7/moving_variance:0', False)]
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_7 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_26 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_8 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 den

In [71]:
# Example applying BN before activation functions
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('elu'),
    tf.keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('elu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

print([(var.name, var.trainable) for var in model.layers[1].variables])
model.summary()

[('batch_normalization_10/gamma:0', True), ('batch_normalization_10/beta:0', True), ('batch_normalization_10/moving_mean:0', False), ('batch_normalization_10/moving_variance:0', False)]
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_10 (Bat  (None, 784)              3136      
 chNormalization)                                                
                                                                 
 dense_29 (Dense)            (None, 300)               235200    
                                                                 
 batch_normalization_11 (Bat  (None, 300)              1200      
 chNormalization)                                                
                                                                 

### Gradient Clipping

Another popular technique to mitigate the exploding gradients problem is to clip the gradients during backpropagation so that they never exceed some threshold. **This is called *Gradient Clipping*. This technique is most often used in recurrent neural networks**, as Batch Normalization is tricky to use in RNNs. For other types of networks, BN is usually sufficient.

An optimizer with a clipvalue=1.0 will clip every component of the gradient vector to a value between -1.0 and 1.0. For instance, if the original gradient vector is [0.9, 100.0], it points mostly in the direction of the second axis; but once you clip it by value, you get [0.9, 1.0], which points roughly in the diagonal between the two axes. In practice, this approach works well. If you want to ensure that Gradient Clipping does not change the direction of the gradient vector, you should clip by norm by setting clipnorm instead of clipvalue. This will clip the whole gradient if its $l_{2}$ norm is greater than the threshold you picked.

For example, if you set clipnorm=1.0, then the vector [0.9, 100.0] will be clipped to [0.00899964, 0.9999595], preserving its orientation but almost eliminating the first component. You can track the size of the gradients using TensorBoard.

You may want to try both clipping by value and clipping by norm, with different thresholds, and see which option performs best on the validation set.

In [79]:
# In Keras, implementing Gradient Clipping is just a matter of setting the clipvalue or clipnorm argument when creating an optimizer
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse', optimizer=optimizer)

optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss='mse', optimizer=optimizer)

### Reusing Pretrained Layers

It is generally not a good idea to train a very large DNN from scratch: instead, you should always try to find an existing neural network that accomplishes a similar task to the one your are trying to tackle (more on this in Chapter 14) then reuse the lower layers of this network. This technique is called *transfer learning*. It will not only speed up training considerably, but also require significantly less training data.

If the input pictures of your new task don't have the same size as the ones used in the original task, you will usually have to add a preprocessing step to resize them to the size expected by the original model. More generallyl, transfer learning will work best when the inputs have similar low-level features.

The output layer of the original model should usually be replaced because it is most likely not useful at all for the new task, and it may not even have the right number of outputs for the new task.

Similarly, the upper hidden layers of the original model are less likely to be as useful as the lower layers. The more simlar the tasks are, the more layers you want to reuse (starting with the lower layers). For very similar tasks, try keeping all the hidden layers and just replacing the output layer.

Try freezing all the reused layers first (i.e., make their weights non-trainable so that Gradient Descent won't modify them), then train your model and see how it performs. Then try unfreezing one or two of the top hidden layers to let backpropagation tweak them and see if performance improves. **The more training data you have, the more layers you can unfreeze. It is also useful to reduce the learning rate when you unfreeze reused layers: this will avoid wrecking their fine-tuned weights.**

If you still cannot get good performance, and you have little training data, try dropping the top hidden layer(s) and freezing all the remaining hidden layers again. You can iterate until you find the right number of layers to reuse. If you have plenty of training data, you may try replacing the top hidden layers instead of dropping them, and even adding more hidden layers.

### Transfer Learning with Keras

Suppose someone built and trained a Keras model on that set and got reasonably good performance. Let's call this model A. You now want to tackle a different task. Your dataset is quite small; only 200 labeled images. When you train a new model for this task (call it model B) with the same architecture as model A, it performs reasonably well (97.2% accuracy). But since it's a much easier task, you were hoping for me. So perhaps transfer learning can help. 

First, you need to load model A and create a new model based on that model's layers. When you train model_B_on_A, it will also affect model_A. If you want to avoid that, you need to *clone* model_A before you reuse its layers. To do this, you clone model A's architecture with clone_model(), then copy its weights (since clone_model() does not clone the weights).

Now you could train model_B_on_A for task B, but since the new output layer was initialized randomly it will make large errors, so there will be large error gradients that may wreck the reused weights. To avoid this, once approach is to freeze the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights. To do this, set every layer's trainable attribute to False and compile the model.

**Note that you must always compile your model after you freeze or unfreeze layers.**

Now you can train the model for a few epochs, then unfreeze the reused layers (which requires compiling the model again) and continue training to fine-tune the reused layers for task B. After unfreezing the reused layers, it is usually a good idea to reduce the learning rate, once again to avoid damaging the reused weights.

A good rule of thumb is when a paper looks too positive, you should be suspicious: perhaps the flashy new technique does not actually help much but the authors tried many variants and reported only the best results without mentioning how many failures they encountered on the way. In the example below, the author suggests the test accuracy is 99.25%! However, it turns out that transfer learning does not work very well with small dense networks, presumably because small networks learn few patterns, and dense networks learn very specific patterns, which are unlikely to be useful in other tasks.

In this case, the author specifically worked to find a set of conditions that led to model_A_on_B working extremely well. However, transfer learning works best with deep convolutional neural networks, which tend to learn feature detectors that are much more general (especially in the lower layers). This concept will be revisited in chapter 14.

In [86]:
'''Example of transfer learning. Commented out because model A doesnt exist'''
# model_A = tf.keras.load_model('my_model_A.h5')
# model_A_clone = tf.keras.models.clone_model(model_A)
# model_A_clone.set_weights(model_A.get_weights())
# model_B_on_A = tf.keras.models.Sequential(model_A_clone.layers[:-1])
# model_B_on_A.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False
    
# model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
# history = model_B_on_A.fit(
#     X_train_B,
#     y_train_B,
#     epochs=4,
#     validation_data=(X_valid_B, y_valid_B)
# )

# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = True
    
# optimizer = tf.keras.SGD(lr=1e-4)
# model_B_on_A.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# history = model_B_on_A.fit(
#     X_train_B,
#     y_train_B,
#     epochs=16,
#     validation_data=(X_valid_B, y_valid_B)
# )

'Example of transfer learning. Commented out because model A doesnt exist'

### Unsupervised Pretraining

Suppose you want to tackle a complex task for which you don't have much labeled training data, but unfortunately you cannot find a model trained on a similar task. You may still be able to perform *unsupervised pretraining*. If you can gather plenty of unlabeled training data, you can try to use it to train an unsupervised model, such as an autoencoder or a generative adversarial network. Then you can reuse the lower layers of the autoencoder or the lower layers of the GAN's discriminator, add the output layer for your task on top, and fine-tune the final network using supervised learning.

Unsupervised pretraining (today typically using autoencoders or GAN's rather than RBMs) is still a good option when you have a complex task to solve, no similar model you can reuse, and little labeled training data but plenty of unlabeled training data.

### Pretraining on an Auxiliary Task

If you do not have much labeled training data, one last option is to train a first neural network on an auxiliary task for which you can easily obtain or generate labeled training data, then reuse the lower layers of that network for your actual task. The first neural network's lower layers will learn feature detectors that will likely be reusable by the second neural network.

For example, if you want to build a system to recognize faces, you may only have a few pictures of each individual-- clearly not enough to train a good classifier. Gathering hundreds of pictures of each person would not be practical. You could, however, gather a lot of pictures of random people on the web and train a first neural network to detect whether or not two different pictures feature the same person. Such a network would learn good feature detectors for faces, so reusing its lower layers would allow you to train a good face classifier that uses little training data.

For *natural language processing (NLP)* applications, you can download a corpus of millions of text documents and automatically generate labeled data from it. For example, you could randomly mask out some words and train a model to predict what the missing words are (e.g., it should predict that the missing word in the sentence "What __ you saying?" is probaby "are" or "were"). If you an train a model to reach a good performance on this task, then it will already know quite a lot about language, and you can certainly reuse it for your actual task and fine-tune it on your labeled data.

*Self-supervised learning* is when you a